In [ ]:
import os
import pandas as pd
from pyensembl import EnsemblRelease

In [ ]:
snakefile_path = os.getcwd() + "/../Snakefile"
snakefile_path

In [ ]:
try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args

    snakemake = load_rule_args(
        snakefile = snakefile_path,
        rule_name = 'ppi_preprocess_string',
        root="../",
    )

In [ ]:
release_no = snakemake.params['release']
input_string_db = snakemake.input['db']
output_string_db = snakemake.output['db']

In [ ]:
string_db = pd.read_csv(input_string_db, sep = ' ')

In [ ]:
string_db.head()

In [ ]:
string_db.protein1 = string_db.protein1.str.split('.').str[1]
string_db.protein2 = string_db.protein2.str.split('.').str[1]

In [ ]:
ens = EnsemblRelease(release_no)

In [ ]:
# Test if retrieving protein ID is possible

ens.gene_id_of_protein_id(string_db.protein1[0])

In [ ]:
def convert_to_gene(prot):
    try:
        return ens.gene_id_of_protein_id(prot)
    except:
        return None

In [ ]:
string_db['gene1'] = string_db.protein1.apply(convert_to_gene)

In [ ]:
string_db['gene2'] = string_db.protein2.apply(convert_to_gene)

## Save processed STRING database

In [ ]:
string_db.to_csv(output_string_db, sep = '\t', index=False, compression='gzip')

## Explore dataset

In [ ]:
number_of_interactions = string_db.shape[0]

In [ ]:
string_db.head()

In [ ]:
string_db.dropna().shape[0]

In [ ]:
number_of_interactions

In [ ]:
string_db.query('combined_score > 900')